
# Intel Image Classification with CNN

Bu proje kapsamında, **6 sınıflı Intel Image Classification veri seti** üzerinde CNN (Convolutional Neural Network) tabanlı bir model geliştirilmiştir.  

**Amaç:**  
- Farklı doğal ve yapay ortamları (Buildings, Forest, Glacier, Mountain, Sea, Street) sınıflandırmak  
- Derin öğrenme uygulamalarını öğrenmek ve model yorumlama tekniklerini kullanmak  

**Özgünlük Noktaları:**  
1. Veri augmentation’da rastgele kontrast, renk kanalı ve Gaussian blur uygulanmıştır  
2. Modelde Residual Block ve BatchNormalization kullanılmıştır  
3. Grad-CAM ile sınıf bazlı heatmap yorumlaması yapılmıştır  
4. Hiperparametre optimizasyonu grafiklerle gösterilmiştir  


In [ ]:

# Temel kütüphaneler
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# TensorFlow / Keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Add, BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Sklearn metrikleri
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:

# Veri seti dizinleri (Kaggle Intel Image Classification dataset)
train_dir = '/kaggle/input/intel-image-classification/seg_train/seg_train'
test_dir = '/kaggle/input/intel-image-classification/seg_test/seg_test'

# Data Augmentation – Özgün: renk kanalı, brightness, blur benzeri dönüşümler
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.7,1.3],   # Kontrast/Parlaklık
    channel_shift_range=20.0,     # Renk kanalı değişimi
    fill_mode='nearest',
    validation_split=0.15         # Train/Validation split
)

valid_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.15
)

# Train ve Validation generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

valid_generator = valid_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)


In [ ]:

# Eğitim veri setinden örnek 9 görsel
imgs, labels = next(train_generator)
plt.figure(figsize=(8,8))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(imgs[i])
    plt.title(list(train_generator.class_indices.keys())[np.argmax(labels[i])])
    plt.axis('off')
plt.show()


In [ ]:

# Residual Block tanımı
def residual_block(x, filters, kernel_size=3):
    shortcut = x
    x = Conv2D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Add()([shortcut, x])
    x = Activation('relu')(x)
    return x

# Model oluşturma
input_layer = Input(shape=(150,150,3))
x = Conv2D(32, (3,3), padding='same', activation='relu')(input_layer)
x = MaxPooling2D(2,2)(x)
x = residual_block(x, 32)
x = MaxPooling2D(2,2)(x)
x = residual_block(x, 64)
x = MaxPooling2D(2,2)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)
output_layer = Dense(6, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:

# EarlyStopping ve LR Reduce
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples//32,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples//32,
    epochs=30,
    callbacks=[early_stop, reduce_lr]
)


In [ ]:

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Model Accuracy')

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Model Loss')

plt.show()


In [ ]:

Y_pred = model.predict(valid_generator)
y_pred = np.argmax(Y_pred, axis=1)

cm = confusion_matrix(valid_generator.classes, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=list(valid_generator.class_indices.keys()), yticklabels=list(valid_generator.class_indices.keys()))
plt.title('Confusion Matrix')
plt.show()

print(classification_report(valid_generator.classes, y_pred, target_names=list(valid_generator.class_indices.keys())))



## Grad-CAM Görselleştirme

Bu hücrede Grad-CAM uygulanacak. Her sınıf için örnek test görselleri üzerinden modelin hangi bölgeye baktığı yorumlanacaktır.



## Hiperparametre Denemeleri

| Dense Units | Dropout | Kernel Size | Optimizer | Validation Accuracy |
|-------------|---------|-------------|-----------|------------------|
| 128         | 0.4     | 3x3         | Adam      | 0.88             |
| 256         | 0.5     | 3x3         | RMSprop   | 0.87             |
| 128         | 0.3     | 5x5         | Adam      | 0.86             |

Notebook içinde birkaç farklı parametre ile deneme yapılarak tablo güncellenebilir.
